# 01 - Price Data Exploration

## Objective
This notebook explores the **electricity price (day-ahead)** time series data.

**Key Questions:**
- What is the temporal extent of the data?
- Are there missing values or outliers?
- What are the statistical properties?
- Are there seasonal patterns (hourly, daily, weekly)?
- Are there negative prices (oversupply situations)?
- What is the optimal test period?

**Expected Challenges:**
- Price is the most volatile energy type
- Contains spikes and negative values
- Expected R² lower than other energy types (0.85-0.92)


In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

## 1. Load Data

In [ ]:
# Define paths
data_path = Path('../../data/raw/price_day_ahead_2022-01-01_2024-12-31_hour.csv')

# Load data
df = pd.read_csv(data_path)
print(f"Data loaded: {df.shape}")
df.head(10)

In [ ]:
# Parse datetime
df['datetime'] = pd.to_datetime(df['datetime'])
df.set_index('datetime', inplace=True)
df.sort_index(inplace=True)

# Rename column
df.columns = ['price']

print(f"\nDate range: {df.index.min()} to {df.index.max()}")
print(f"Total hours: {len(df)}")
print(f"Expected hours (3 years): {3*365*24} = {3*365*24}")
df.info()

## 2. Data Quality Checks

In [ ]:
# Missing values
print("Missing values:", df.isnull().sum().sum())

# Zero values
print(f"Zero values: {(df['price'] == 0).sum()}")

# Negative values (important for price!)
negative_count = (df['price'] < 0).sum()
print(f"\n🔴 Negative prices: {negative_count} ({negative_count/len(df)*100:.2f}%)")
if negative_count > 0:
    print(f"   Min price: {df['price'].min():.2f} EUR/MWh")
    print(f"   Negative price dates:")
    print(df[df['price'] < 0].head(10))

In [ ]:
# Statistical summary
print("\n📊 Statistical Summary:")
print(df['price'].describe())
print(f"\nCoefficient of Variation: {df['price'].std() / df['price'].mean():.3f}")
print(f"Skewness: {df['price'].skew():.3f}")
print(f"Kurtosis: {df['price'].kurtosis():.3f}")

## 3. Timeline Visualization

In [ ]:
# Full timeline
fig, ax = plt.subplots(figsize=(16, 5))
ax.plot(df.index, df['price'], linewidth=0.5, alpha=0.7)
ax.axhline(0, color='red', linestyle='--', linewidth=1, label='Zero line')
ax.set_title('Electricity Price (Day-Ahead) - Full Timeline (2022-2024)', fontsize=14, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Price (EUR/MWh)')
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('../../results/figures/price_full_timeline.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Zoom into specific periods
fig, axes = plt.subplots(3, 1, figsize=(16, 10))

# 2022
df_2022 = df['2022']
axes[0].plot(df_2022.index, df_2022['price'], linewidth=0.8, color='steelblue')
axes[0].axhline(0, color='red', linestyle='--', linewidth=1)
axes[0].set_title('2022: Price Timeline', fontweight='bold')
axes[0].set_ylabel('Price (EUR/MWh)')
axes[0].grid(alpha=0.3)

# 2023
df_2023 = df['2023']
axes[1].plot(df_2023.index, df_2023['price'], linewidth=0.8, color='darkorange')
axes[1].axhline(0, color='red', linestyle='--', linewidth=1)
axes[1].set_title('2023: Price Timeline', fontweight='bold')
axes[1].set_ylabel('Price (EUR/MWh)')
axes[1].grid(alpha=0.3)

# 2024
df_2024 = df['2024']
axes[2].plot(df_2024.index, df_2024['price'], linewidth=0.8, color='seagreen')
axes[2].axhline(0, color='red', linestyle='--', linewidth=1)
axes[2].set_title('2024: Price Timeline', fontweight='bold')
axes[2].set_ylabel('Price (EUR/MWh)')
axes[2].set_xlabel('Date')
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('../../results/figures/price_yearly_timelines.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. Distribution Analysis

In [ ]:
# Distribution plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(df['price'], bins=100, edgecolor='black', alpha=0.7)
axes[0].axvline(df['price'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {df["price"].mean():.2f}')
axes[0].axvline(df['price'].median(), color='green', linestyle='--', linewidth=2, label=f'Median: {df["price"].median():.2f}')
axes[0].axvline(0, color='black', linestyle='-', linewidth=1, label='Zero')
axes[0].set_title('Price Distribution', fontweight='bold')
axes[0].set_xlabel('Price (EUR/MWh)')
axes[0].set_ylabel('Frequency')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Box plot
axes[1].boxplot(df['price'], vert=True)
axes[1].axhline(0, color='red', linestyle='--', linewidth=1)
axes[1].set_title('Price Box Plot', fontweight='bold')
axes[1].set_ylabel('Price (EUR/MWh)')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('../../results/figures/price_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. Temporal Patterns

In [ ]:
# Add time features
df['hour'] = df.index.hour
df['day_of_week'] = df.index.dayofweek  # 0=Monday, 6=Sunday
df['month'] = df.index.month
df['year'] = df.index.year

In [ ]:
# Hourly pattern
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# By hour
hourly_avg = df.groupby('hour')['price'].mean()
hourly_std = df.groupby('hour')['price'].std()
axes[0].plot(hourly_avg.index, hourly_avg.values, marker='o', linewidth=2, markersize=6)
axes[0].fill_between(hourly_avg.index, 
                       hourly_avg.values - hourly_std.values, 
                       hourly_avg.values + hourly_std.values, 
                       alpha=0.3)
axes[0].set_title('Average Price by Hour of Day', fontweight='bold')
axes[0].set_xlabel('Hour')
axes[0].set_ylabel('Average Price (EUR/MWh)')
axes[0].grid(alpha=0.3)
axes[0].set_xticks(range(24))

# By day of week
dow_avg = df.groupby('day_of_week')['price'].mean()
dow_std = df.groupby('day_of_week')['price'].std()
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
axes[1].bar(range(7), dow_avg.values, alpha=0.7, edgecolor='black')
axes[1].errorbar(range(7), dow_avg.values, yerr=dow_std.values, fmt='none', color='red', capsize=5)
axes[1].set_title('Average Price by Day of Week', fontweight='bold')
axes[1].set_xlabel('Day of Week')
axes[1].set_ylabel('Average Price (EUR/MWh)')
axes[1].set_xticks(range(7))
axes[1].set_xticklabels(days)
axes[1].grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('../../results/figures/price_temporal_patterns.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Seasonal pattern
fig, ax = plt.subplots(figsize=(12, 5))
monthly_avg = df.groupby('month')['price'].mean()
monthly_std = df.groupby('month')['price'].std()
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
ax.bar(range(1, 13), monthly_avg.values, alpha=0.7, edgecolor='black')
ax.errorbar(range(1, 13), monthly_avg.values, yerr=monthly_std.values, fmt='none', color='red', capsize=5)
ax.set_title('Average Price by Month', fontweight='bold')
ax.set_xlabel('Month')
ax.set_ylabel('Average Price (EUR/MWh)')
ax.set_xticks(range(1, 13))
ax.set_xticklabels(months)
ax.grid(alpha=0.3, axis='y')
plt.tight_layout()
plt.savefig('../../results/figures/price_seasonal_pattern.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Autocorrelation Analysis

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# ACF
plot_acf(df['price'].dropna(), lags=168, ax=axes[0])  # 168 hours = 1 week
axes[0].set_title('Autocorrelation Function (ACF)', fontweight='bold')
axes[0].set_xlabel('Lag (hours)')

# PACF
plot_pacf(df['price'].dropna(), lags=168, ax=axes[1])
axes[1].set_title('Partial Autocorrelation Function (PACF)', fontweight='bold')
axes[1].set_xlabel('Lag (hours)')

plt.tight_layout()
plt.savefig('../../results/figures/price_autocorrelation.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. Volatility and Spikes Analysis

In [ ]:
# Identify extreme values (spikes)
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 3 * IQR
upper_bound = Q3 + 3 * IQR

spikes_low = df[df['price'] < lower_bound]
spikes_high = df[df['price'] > upper_bound]

print(f"Lower bound: {lower_bound:.2f} EUR/MWh")
print(f"Upper bound: {upper_bound:.2f} EUR/MWh")
print(f"\nLow spikes (< {lower_bound:.2f}): {len(spikes_low)} ({len(spikes_low)/len(df)*100:.2f}%)")
print(f"High spikes (> {upper_bound:.2f}): {len(spikes_high)} ({len(spikes_high)/len(df)*100:.2f}%)")
print(f"Total spikes: {len(spikes_low) + len(spikes_high)} ({(len(spikes_low) + len(spikes_high))/len(df)*100:.2f}%)")

In [ ]:
# Visualize spikes
fig, ax = plt.subplots(figsize=(16, 5))
ax.plot(df.index, df['price'], linewidth=0.5, alpha=0.5, label='Price')
ax.scatter(spikes_low.index, spikes_low['price'], color='blue', s=20, label=f'Low spikes ({len(spikes_low)})', zorder=5)
ax.scatter(spikes_high.index, spikes_high['price'], color='red', s=20, label=f'High spikes ({len(spikes_high)})', zorder=5)
ax.axhline(upper_bound, color='red', linestyle='--', linewidth=1, alpha=0.5)
ax.axhline(lower_bound, color='blue', linestyle='--', linewidth=1, alpha=0.5)
ax.axhline(0, color='black', linestyle='-', linewidth=1)
ax.set_title('Price Spikes Detection (3×IQR method)', fontweight='bold', fontsize=14)
ax.set_xlabel('Date')
ax.set_ylabel('Price (EUR/MWh)')
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('../../results/figures/price_spikes.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Rolling volatility
df['rolling_std_24h'] = df['price'].rolling(window=24).std()
df['rolling_std_168h'] = df['price'].rolling(window=168).std()  # 1 week

fig, ax = plt.subplots(figsize=(16, 5))
ax.plot(df.index, df['rolling_std_24h'], linewidth=1, label='24h Rolling Std', alpha=0.7)
ax.plot(df.index, df['rolling_std_168h'], linewidth=1.5, label='168h (1 week) Rolling Std', alpha=0.7)
ax.set_title('Price Volatility Over Time', fontweight='bold', fontsize=14)
ax.set_xlabel('Date')
ax.set_ylabel('Standard Deviation (EUR/MWh)')
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('../../results/figures/price_volatility.png', dpi=150, bbox_inches='tight')
plt.show()

## 8. Test Period Selection

In [ ]:
# Recommendation: Use last 3 months as test (similar to other energy types)
test_start = '2024-10-01'
val_start = '2024-07-01'

train = df[:val_start]
val = df[val_start:test_start]
test = df[test_start:]

print("📊 Dataset Split:")
print(f"Train: {train.index.min()} to {train.index.max()} ({len(train)} hours)")
print(f"Val:   {val.index.min()} to {val.index.max()} ({len(val)} hours)")
print(f"Test:  {test.index.min()} to {test.index.max()} ({len(test)} hours)")
print(f"\nTotal: {len(df)} hours")
print(f"Train: {len(train)/len(df)*100:.1f}%")
print(f"Val:   {len(val)/len(df)*100:.1f}%")
print(f"Test:  {len(test)/len(df)*100:.1f}%")

In [ ]:
# Visualize split
fig, ax = plt.subplots(figsize=(16, 5))
ax.plot(train.index, train['price'], linewidth=0.8, label='Train', alpha=0.7)
ax.plot(val.index, val['price'], linewidth=0.8, label='Validation', alpha=0.7)
ax.plot(test.index, test['price'], linewidth=0.8, label='Test', alpha=0.7)
ax.axvline(pd.to_datetime(val_start), color='orange', linestyle='--', linewidth=2, label='Val start')
ax.axvline(pd.to_datetime(test_start), color='red', linestyle='--', linewidth=2, label='Test start')
ax.axhline(0, color='black', linestyle='-', linewidth=1)
ax.set_title('Train/Val/Test Split', fontweight='bold', fontsize=14)
ax.set_xlabel('Date')
ax.set_ylabel('Price (EUR/MWh)')
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('../../results/figures/price_train_val_test_split.png', dpi=150, bbox_inches='tight')
plt.show()

## 9. Key Findings Summary

In [ ]:
print("="*80)
print("📋 PRICE DATA EXPLORATION - KEY FINDINGS")
print("="*80)
print(f"\n1. DATA COMPLETENESS:")
print(f"   - Date range: {df.index.min()} to {df.index.max()}")
print(f"   - Total hours: {len(df)}")
print(f"   - Missing values: {df.isnull().sum().sum()}")

print(f"\n2. STATISTICAL PROPERTIES:")
print(f"   - Mean: {df['price'].mean():.2f} EUR/MWh")
print(f"   - Median: {df['price'].median():.2f} EUR/MWh")
print(f"   - Std Dev: {df['price'].std():.2f} EUR/MWh")
print(f"   - CV: {df['price'].std() / df['price'].mean():.3f}")
print(f"   - Min: {df['price'].min():.2f} EUR/MWh")
print(f"   - Max: {df['price'].max():.2f} EUR/MWh")

print(f"\n3. SPECIAL CHARACTERISTICS:")
negative_count = (df['price'] < 0).sum()
print(f"   - Negative prices: {negative_count} ({negative_count/len(df)*100:.2f}%)")
print(f"   - Zero prices: {(df['price'] == 0).sum()}")
print(f"   - High spikes (>{upper_bound:.0f}): {len(spikes_high)}")
print(f"   - Low spikes (<{lower_bound:.0f}): {len(spikes_low)}")

print(f"\n4. TEMPORAL PATTERNS:")
print(f"   - Peak hour: {hourly_avg.idxmax()}:00 ({hourly_avg.max():.2f} EUR/MWh)")
print(f"   - Low hour: {hourly_avg.idxmin()}:00 ({hourly_avg.min():.2f} EUR/MWh)")
print(f"   - Weekday avg: {df[df['day_of_week'] < 5]['price'].mean():.2f} EUR/MWh")
print(f"   - Weekend avg: {df[df['day_of_week'] >= 5]['price'].mean():.2f} EUR/MWh")

print(f"\n5. DATASET SPLIT:")
print(f"   - Train: {len(train)} hours ({len(train)/len(df)*100:.1f}%)")
print(f"   - Val:   {len(val)} hours ({len(val)/len(df)*100:.1f}%)")
print(f"   - Test:  {len(test)} hours ({len(test)/len(df)*100:.1f}%)")

print(f"\n6. MODELING EXPECTATIONS:")
print(f"   - Expected R² range: 0.85 - 0.92 (most challenging energy type)")
print(f"   - Key challenges: Volatility, spikes, negative prices")
print(f"   - Best model expected: LightGBM or XGBoost")
print(f"   - Special handling needed: Negative price detection/treatment")

print("\n" + "="*80)
print("✅ Data exploration complete! Ready for preprocessing.")
print("="*80)

## Next Steps

1. ✅ Data exploration complete
2. ➡️ **Next:** `02_price_preprocessing.ipynb`
   - Handle negative prices (keep them? log transform?)
   - Feature engineering (46 features)
   - Train/Val/Test split
   - Scaling
3. 📊 Then: Baseline, Statistical, ML, and Deep Learning models